# Determine which interactions are key to generating the stable states

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from functools import reduce
import random

In [632]:
import pickle
from collections import Counter
from itertools import combinations
import interactions
import initialize
from util import action_set, to_action, format_actions
from action import Action

In [4]:
from segment_polarity_model import SegmentPolarityModel
from tree_search import SearchState
import attractors

In [273]:
from joblib import Parallel, delayed

In [595]:
datadir='/home/bglaze/segment_polarity/data'
tmstp='20210415-1427'
f = open(f'{datadir}/models/best_models/best_models_clusters_tmstp-{tmstp}.pickle','rb')

clusters = pickle.load(f)

In [256]:
# Remove all interactions corresponding to a specific action from the rules of a given model
def knockout_interactions(model, knockouts):
    if type(knockouts) is Action:
        raise TypeError(f'knockouts argument must be a list of Actions, got {type(knockouts)}')
    
    knockout_model = dict()
    
    for dst, (act, inh) in model.rules.items():
        knockout_model[dst] = ([], []) 
        dst_node, dst_cell = dst
        for src in act:
            action = to_action(src,dst,'a')
            if action not in knockouts:
                knockout_model[dst][0].append(src)
        for src in inh:
            action = to_action(src,dst,'i')
            if action not in knockouts:
                knockout_model[dst][1].append(src)
    return SegmentPolarityModel(knockout_model)

### Common Interactions

In [9]:
predefined,_ = initialize.initialState()

predefined = action_set(predefined.rules)

In [644]:
predefined

{Action(srcs=(('CI', 'i'),), dst='CIA', type='a'),
 Action(srcs=(('CI', 'i'),), dst='CIR', type='a'),
 Action(srcs=(('HH', 'e'),), dst='CIR', type='i'),
 Action(srcs=(('PTC', 'i'), ('HH', 'e')), dst='PH', type='a'),
 Action(srcs=(('PTC', 'i'), ('~HH', 'e')), dst='PTC', type='a'),
 Action(srcs=(('WG', 'e'),), dst='en', type='a'),
 Action(srcs=(('ci', 'i'),), dst='CI', type='a'),
 Action(srcs=(('en', 'i'),), dst='EN', type='a'),
 Action(srcs=(('hh', 'i'),), dst='HH', type='a'),
 Action(srcs=(('ptc', 'i'),), dst='PTC', type='a'),
 Action(srcs=(('wg', 'i'),), dst='WG', type='a'),
 Action(srcs=(('~PTC', 'i'),), dst='SMO', type='a')}

In [297]:
import reference_model

In [298]:
reference_actions = set(reference_model.actions)

In [10]:
all_models = []
for cluster, models in clusters.items():
    all_models += models
print(len(all_models))

50000


In [102]:
common_interactions = interactions.common(all_models, order=[1], verbose=True)

common analysis progress: 10%
common analysis progress: 20%
common analysis progress: 30%
common analysis progress: 40%
common analysis progress: 50%
common analysis progress: 60%
common analysis progress: 70%
common analysis progress: 80%
common analysis progress: 90%
common analysis progress: 100%


In [456]:
header = '% models | matches reference? | interaction'

i=0
print(header)
for actions,pct in sorted(list(common_interactions.items()), key=lambda _:_[1], reverse=True):
    action = actions[0]
    if action not in predefined: 
        print(f'{pct*100:3.0f}% {"x" if action in reference_actions else " "}', end=' ')
        format_actions(actions)
        if i==50:
            break
        i+=1
        

% models | matches reference? | interaction
100% x (EN_i inhibit ci) 
100%   (CIR_i inhibit CIA) 
 85% x (SLP_i inhibit en) 
 74% x (CIA_i and SLP_i activate wg) 
 72%   (EN_i inhibit CI) 
 59% x (CIA_i activate ptc) 
 51%   (CI_i inhibit hh) 
 51% x (EN_i activate hh) 
 43%   (CIR_i inhibit WG) 
 41%   (CI_i and SMO_i activate ptc) 
 41% x (CIR_i inhibit wg) 
 40% x (HH_{i+-1} activate SMO) 
 37%   (SLP_i inhibit hh) 
 35%   (EN_i activate HH) 
 34% x (SLP_i and wg_i activate wg) 
 34%   (CIA_i activate SMO) 
 34%   (CIR_i inhibit SMO) 
 32%   (CIA_i activate PTC) 
 27%   (EN_i inhibit PTC) 
 27%   (WG_i activate SMO) 
 26%   (WG_i activate CI) 
 25%   (CIA_i and SLP_i activate CI) 
 25%   (PH_i inhibit EN) 
 25%   (EN_i inhibit wg) 
 24%   (WG_i activate CIA) 
 24%   (PTC_i inhibit HH) 
 23%   (PH_i activate SMO) 
 23%   (PTC_i activate CIR) 
 23% x (CI_i and SMO_i activate CIA) 
 22% x (EN_i inhibit ptc) 
 22%   (PH_i activate PTC) 
 22%   (HH_i activate SMO) 
 21%   (CIA_i activate

### Interactions unique to each cluster

#### First order (individual interactions)

In [127]:
unique_interactions = interactions.unique(clusters,
                                          uniqueness_threshold=.95, 
                                          order=[1],
                                          verbose=False)

In [128]:
for cluster in unique_interactions:
    print(cluster, len(unique_interactions[cluster]))

0 3
1 1
2 2
3 1
4 0
5 0
6 0
7 1
8 2
9 1
10 2


#### Higher order (n-tuples of interactions)

In [267]:
unique_interactions = interactions.unique(clusters,
                                          uniqueness_threshold=.90, 
                                          order=[1,2,3,4,5],
                                          verbose=True,
                                          excluded=predefined
                                         )

Common analysis cluster: 0
common analysis progress: 10%
common analysis progress: 20%
common analysis progress: 30%
common analysis progress: 40%
common analysis progress: 50%
common analysis progress: 60%
common analysis progress: 70%
common analysis progress: 80%
common analysis progress: 90%
common analysis progress: 100%
Common analysis cluster: 1
common analysis progress: 10%
common analysis progress: 20%
common analysis progress: 30%
common analysis progress: 40%
common analysis progress: 50%
common analysis progress: 60%
common analysis progress: 70%
common analysis progress: 80%
common analysis progress: 90%
common analysis progress: 100%
Common analysis cluster: 2
common analysis progress: 10%
common analysis progress: 20%
common analysis progress: 30%
common analysis progress: 40%
common analysis progress: 50%
common analysis progress: 60%
common analysis progress: 70%
common analysis progress: 80%
common analysis progress: 90%
common analysis progress: 100%
Common analysis 

In [270]:
for cluster in unique_interactions:
    print(cluster, len(unique_interactions[cluster]))

0 8830
1 54
2 150
3 41
4 25
5 164
6 36
7 132
8 513
9 67
10 21570


In [271]:
filtered_interactions = {}

for cluster in unique_interactions:
    filtered_interactions[cluster] = interactions.superset_filter(unique_interactions[cluster])
    print(cluster, len(filtered_interactions[cluster]))

0 31
1 12
2 12
3 3
4 6
5 18
6 6
7 19
8 13
9 6
10 57


In [607]:
for cluster in filtered_interactions:
    print(f'Cluster {cluster}')
    print('=====================')
    for actions,percent in Counter(unique_interactions[cluster]).most_common(10):
        print(f'{percent:>6.4}', end=' ')
        format_actions(actions)    

Cluster 0
   1.0 (WG_{i+-1} activate PH) + (CIR_i inhibit WG) 
   1.0 (SMO_i activate PH) + (CI_i inhibit en) 
   1.0 (EN_i inhibit ci) + (WG_{i+-1} activate PH) + (CIR_i inhibit WG) 
   1.0 (WG_{i+-1} activate PH) + (CIR_i inhibit PH) + (CIR_i inhibit WG) 
   1.0 (SLP_i activate WG) + (CIA_i activate SMO) + (SMO_i activate PH) 
   1.0 (SLP_i activate WG) + (CIA_i and SLP_i activate CI) + (CI_i inhibit en) 
   1.0 (SLP_i activate WG) + (SMO_i activate PH) + (CI_i inhibit en) 
   1.0 (CIA_i activate SMO) + (CIA_i and SLP_i activate CI) + (SMO_i activate PH) 
   1.0 (CIA_i activate SMO) + (EN_i activate HH) + (SMO_i activate PH) 
   1.0 (CIA_i activate SMO) + (SMO_i activate PH) + (CI_i inhibit en) 
Cluster 1
0.9962 (CIA_i activate ptc) + (CIR_i inhibit wg) + (EN_i inhibit CI) + (CIR_i inhibit WG) 
0.9962 (CIA_i activate ptc) + (CIR_i inhibit CIA) + (CIR_i inhibit wg) + (EN_i inhibit CI) + (CIR_i inhibit WG) 
0.9946 (CIR_i inhibit wg) + (EN_i inhibit CI) + (CIR_i inhibit WG) 
0.9946 (CIR

### Knockouts of common interactions

In [490]:
def simulate_knockouts(idx, interactions, models, verbose=False):
    n_models = len(models)

    knockout_simulations = {action:[0,0] for action in interactions}
    
    step = int(n_models/10)
    step = 1 if step == 0 else step
    
    i=0
    for model in models:
        if i%step==0 and verbose:
            print(f'{i/n_models*100:.0f}%')
        individual_cache = dict()

        for knockouts in interactions:
            individual_scores = []

            ko = SegmentPolarityModel(rules = model)
            ko = knockout_interactions(ko, knockouts)
            ko.compile_rules()
            state = SearchState(ko, 0,0,0,None)
            combined_score = state.getReward()

            if len(knockouts) > 1:
                for knockout in knockouts:
                    if knockout in individual_cache:
                        individual_scores.append(individual_cache[knockout])
                    else:
                        ko = SegmentPolarityModel(rules = model)
                        ko = knockout_interactions(ko, [knockout])
                        ko.compile_rules()
                        state = SearchState(ko, 0,0,0,None)
                        score = state.getReward()
                        individual_scores.append(score)
                        individual_cache[knockout] = score
            else:
                individual_scores.append(combined_score)

            knockout_simulations[knockouts][0] += combined_score/n_models
            knockout_simulations[knockouts][1] += (min(individual_scores) - combined_score)/n_models
        i+=1
    return (idx, knockout_simulations)

In [619]:
most_common_actions = []
sorted_common_actions = sorted(list(common_interactions.items()), key=lambda _:_[1], reverse=True)
for actions,pct in sorted_common_actions:
    # NOTE this only works when we're using order=[1]
    action = actions[0]
    if action not in predefined:
        if pct > .95:
            print(pct, end=" ")
            format_actions(actions)
            most_common_actions.append(actions)
        else:
            break
    

1.0000000000007185 (EN_i inhibit ci) 
1.0000000000007185 (CIR_i inhibit CIA) 


In [367]:
combined_common_interactions = tuple([action[0] for action in most_common_actions])

In [373]:
_, common_knockout_scores = simulate_knockouts(0, [combined_common_interactions], random.sample(all_models, k=1000))

0%


In [631]:
common_sorted_scores = sorted(common_knockout_scores.items(), key=lambda _:_[1])

for actions, (combined_score, synergy_score) in common_sorted_scores:
    print(f'{1-combined_score:.4f} {synergy_score:.4f}', end = " ")
    format_actions(actions)


0.3113 0.0169 (EN_i inhibit ci) + (CIR_i inhibit CIA) 


### Comparison to knockouts of less common interactions

In [390]:
action_to_model = dict()
for i in range(len(all_models)):
    model = all_models[i]
    actions = action_set(model)
    for action in actions:
        if action in predefined:
            continue
        if action not in action_to_model: action_to_model[action] = list()
        action_to_model[action].append(i)

In [437]:
common_knockdown_scores = {}
for action, idxs in sorted(list(action_to_model.items()), key=lambda _:len(_[1]), reverse=True):
    sampled_models = []
    for idx in random.sample(idxs, k=min(len(idxs), 100)):
        sampled_models.append(all_models[idx])
    
    _, knockout_score = simulate_knockouts(0, [(action,)], sampled_models)
    _,(combined_score, _) = list(knockout_score.items())[0]
    common_knockdown_scores[action] = combined_score

In [630]:
for action, score in common_knockdown_scores.items():
    print(f'{1-score:.4f}', end=' ')
    format_actions((action,))

0.2861 (EN_i inhibit ci) 
0.1426 (CIR_i inhibit CIA) 
0.0265 (SLP_i inhibit en) 
0.0610 (CIA_i and SLP_i activate wg) 
0.0668 (EN_i inhibit CI) 
0.0342 (CIA_i activate ptc) 
0.0686 (CI_i inhibit hh) 
0.1004 (EN_i activate hh) 
0.0281 (CIR_i inhibit WG) 
0.0489 (CI_i and SMO_i activate ptc) 
0.0437 (CIR_i inhibit wg) 
0.0484 (HH_{i+-1} activate SMO) 
0.0023 (SLP_i inhibit hh) 
0.0131 (EN_i activate HH) 
0.0747 (SLP_i and wg_i activate wg) 
0.0087 (CIA_i activate SMO) 
0.0389 (CIR_i inhibit SMO) 
0.0007 (CIA_i activate PTC) 
0.0416 (EN_i inhibit PTC) 
-0.0000 (WG_i activate SMO) 
-0.0000 (WG_i activate CI) 
-0.0000 (CIA_i and SLP_i activate CI) 
0.0068 (PH_i inhibit EN) 
-0.0000 (EN_i inhibit wg) 
-0.0000 (WG_i activate CIA) 
0.1875 (PTC_i inhibit HH) 
0.0062 (PH_i activate SMO) 
0.0006 (PTC_i activate CIR) 
-0.0000 (CI_i and SMO_i activate CIA) 
0.0108 (EN_i inhibit ptc) 
0.0002 (PH_i activate PTC) 
-0.0000 (HH_i activate SMO) 
-0.0000 (CIA_i activate CIR) 
0.0092 (CIA_i activate CI) 
0

### Knockout interactions that are unique to a given cluster

Then simulate for each model in that cluster and determine how badly that destabilizes the behavior of those models.

<!-- We do this for combinations of unique interactions as well -->

### All clusters, unfiltered

In [597]:
# Split up the groups of interactions and models into evenly spaced groups
calls = []
n_samples = 100
step = 100
for cluster in clusters:
    # min call is to ensure that we get at most n_samples models.
    # If the cluster has less than n_samples models in it, then we'll just get all of them
    models = random.sample(clusters[cluster], k=(min(n_samples, len(clusters[cluster]))))
    interactions = list(unique_interactions[cluster].keys())
    for start in range(0, len(interactions), step):
        calls.append(
                delayed(simulate_knockouts)(
                    cluster, 
                    interactions[start:start+step], 
                    models)
                )

In [598]:
with Parallel(n_jobs=45, verbose=10) as parallel:
    parallel_knockout_simulations = parallel(calls)    

[Parallel(n_jobs=45)]: Using backend LokyBackend with 45 concurrent workers.
[Parallel(n_jobs=45)]: Done   8 tasks      | elapsed:  3.1min
[Parallel(n_jobs=45)]: Done  23 tasks      | elapsed:  3.2min
[Parallel(n_jobs=45)]: Done  38 tasks      | elapsed:  3.3min
[Parallel(n_jobs=45)]: Done  55 tasks      | elapsed:  5.7min
[Parallel(n_jobs=45)]: Done  72 tasks      | elapsed:  6.3min
[Parallel(n_jobs=45)]: Done  91 tasks      | elapsed:  6.5min
[Parallel(n_jobs=45)]: Done 110 tasks      | elapsed:  9.3min
[Parallel(n_jobs=45)]: Done 131 tasks      | elapsed:  9.6min
[Parallel(n_jobs=45)]: Done 152 tasks      | elapsed: 12.1min
[Parallel(n_jobs=45)]: Done 175 tasks      | elapsed: 12.8min
[Parallel(n_jobs=45)]: Done 198 tasks      | elapsed: 14.8min
[Parallel(n_jobs=45)]: Done 223 tasks      | elapsed: 16.0min
[Parallel(n_jobs=45)]: Done 266 out of 322 | elapsed: 19.1min remaining:  4.0min
[Parallel(n_jobs=45)]: Done 299 out of 322 | elapsed: 21.0min remaining:  1.6min
[Parallel(n_jobs=

In [599]:
# Merge together the split up simulations
knockout_simulations = {}
for key,scores in parallel_knockout_simulations:
    if key not in knockout_simulations: knockout_simulations[key] = {}
    # this is a weird bit of code but it just merges two dictionaries
    knockout_simulations[key] = {**knockout_simulations[key], **scores}

In [645]:
action_counts = {}

for (cluster, simulation_scores) in knockout_simulations.items():
    print('Cluster', cluster)
    print('===================')
  
    # sort by the combined score first
    sorted_scores = sorted(simulation_scores.items(), key=lambda _:_[1])[:10]
    
    for actions, (combined_score, synergy_score) in sorted_scores:
        print(f'{1-combined_score:.4f} {synergy_score:.4f}', end = " ")
        format_actions(sorted(actions))
        for action in actions:
            if action not in action_counts: action_counts[action] = []
            action_counts[action].append(cluster)

Cluster 0
0.3655 0.0894 (CI_i inhibit en) + (CIR_i inhibit PH) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (PH_i activate PTC) 
0.3649 0.0888 (CI_i inhibit en) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (HH_i inhibit PH) + (PH_i activate PTC) 
0.3643 0.0882 (CI_i inhibit en) + (CIA_i and SLP_i activate CI) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (PH_i activate PTC) 
0.3613 0.0852 (CI_i inhibit en) + (CI_i and SMO_i activate ptc) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (PH_i activate PTC) 
0.3608 0.0846 (CIA_i and SLP_i activate CI) + (CIR_i inhibit CIA) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (PH_i activate PTC) 
0.3601 0.0840 (CI_i inhibit en) + (CIR_i inhibit WG) + (EN_i activate HH) + (EN_i inhibit ci) + (PH_i activate PTC) 
0.3585 0.0864 (CIA_i and SLP_i activate CI) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (EN_i activate hh) + (PH_i activate PTC) 
0.3565 0.0804 (CI_i inhibit en) + (CIR_i inhibit WG) + (EN_i inhibit ci) + (PH_i activate PTC) + (WG_{i+-1} activate PH) 
0.3

In [661]:
max_disruptors = []

for (cluster, simulation_scores) in knockout_simulations.items():
    # sort by the combined score first
    max_disruptor, scores = min(simulation_scores.items(), key=lambda _:_[1])
    max_disruptors.append(set(max_disruptor))

In [666]:
from statistics import mean, median

### Maximally disruptive sets are disjoint

The sets of interactions that disrupt model behavior don't have many interactions in common

In [670]:
max_disruptor_similarities = []

for a,b in combinations(max_disruptors, r=2):
#     format_actions(a)
#     format_actions(b)
    print(f'inter: {len(a & b)}, union: {len(a|b)}, jaccard: {len(a & b)/len(a|b)}')
    max_disruptor_similarities.append(len(a & b)/len(a|b))
#     print('-')

print(max(max_disruptor_similarities))
print(mean(max_disruptor_similarities))
print(median(max_disruptor_similarities))

inter: 2, union: 7, jaccard: 0.2857142857142857
inter: 1, union: 9, jaccard: 0.1111111111111111
inter: 1, union: 8, jaccard: 0.125
inter: 1, union: 8, jaccard: 0.125
inter: 1, union: 9, jaccard: 0.1111111111111111
inter: 1, union: 8, jaccard: 0.125
inter: 1, union: 8, jaccard: 0.125
inter: 1, union: 9, jaccard: 0.1111111111111111
inter: 0, union: 9, jaccard: 0.0
inter: 1, union: 9, jaccard: 0.1111111111111111
inter: 2, union: 7, jaccard: 0.2857142857142857
inter: 2, union: 6, jaccard: 0.3333333333333333
inter: 3, union: 5, jaccard: 0.6
inter: 3, union: 6, jaccard: 0.5
inter: 2, union: 6, jaccard: 0.3333333333333333
inter: 2, union: 6, jaccard: 0.3333333333333333
inter: 1, union: 8, jaccard: 0.125
inter: 0, union: 8, jaccard: 0.0
inter: 2, union: 7, jaccard: 0.2857142857142857
inter: 2, union: 7, jaccard: 0.2857142857142857
inter: 2, union: 7, jaccard: 0.2857142857142857
inter: 2, union: 8, jaccard: 0.25
inter: 2, union: 7, jaccard: 0.2857142857142857
inter: 3, union: 6, jaccard: 0.5
in

In [649]:
action_counts = {}
cluster=7

simulation_scores = knockout_simulations[7]
print('Cluster', cluster)
print('===================')

# sort by the combined score first
sorted_scores = sorted(simulation_scores.items(), key=lambda _:_[1])[-20:]

for actions, (combined_score, synergy_score) in sorted_scores:
    print(f'{combined_score:.4f} {synergy_score:.4f}', end = " ")
    format_actions(sorted(actions))
    for action in actions:
        if action not in action_counts: action_counts[action] = []
        action_counts[action].append(cluster)

Cluster 7
0.9606 0.0000 (CI_i and SMO_i activate CIA) + (HH_i inhibit PTC) 
0.9770 0.0081 (CI_i and SMO_i activate ptc) + (CIR_i inhibit SMO) + (HH_{i+-1} activate PTC) 
0.9770 0.0081 (CI_i and SMO_i activate CIA) + (CI_i and SMO_i activate ptc) + (CIR_i inhibit SMO) + (HH_{i+-1} activate PTC) 
0.9787 0.0076 (CI_i and SMO_i activate ptc) + (HH_{i+-1} activate PTC) 
0.9787 0.0076 (CI_i and SMO_i activate CIA) + (CI_i and SMO_i activate ptc) + (HH_{i+-1} activate PTC) 
0.9846 0.0004 (CI_i and SMO_i activate ptc) + (CIR_i inhibit SMO) 
0.9846 0.0004 (CI_i and SMO_i activate CIA) + (CI_i and SMO_i activate ptc) + (CIR_i inhibit SMO) 
0.9863 0.0000 (CI_i and SMO_i activate CIA) + (CI_i and SMO_i activate ptc) 
0.9957 0.0003 (CIR_i inhibit SMO) + (CIR_i inhibit ptc) + (HH_{i+-1} activate PTC) 
0.9957 0.0003 (CI_i and SMO_i activate CIA) + (CIR_i inhibit SMO) + (CIR_i inhibit ptc) + (HH_{i+-1} activate PTC) 
0.9958 0.0002 (CIR_i inhibit SMO) + (CIR_i inhibit ptc) 
0.9958 0.0002 (CI_i and SMO_

In [618]:
for action, cluster_idxs in sorted(action_counts.items(), key=lambda _:len(_[1]), reverse=True):
    print(f'{len(cluster_idxs):3d}', end = ' ')
    format_actions((action,))


 85 (EN_i inhibit ci) 
 40 (SLP_i inhibit en) 
 25 (CIR_i inhibit CIA) 
 24 (EN_i activate hh) 
 24 (SLP_i and wg_i activate wg) 
 16 (CIR_i inhibit WG) 
 13 (CIR_i inhibit ptc) 
 11 (SLP_i activate CIA) 
 11 (PH_i activate SMO) 
 11 (SLP_i inhibit hh) 
 11 (CIA_i and SLP_i activate wg) 
 10 (PH_i activate PTC) 
 10 (EN_i inhibit ptc) 
 10 (CIA_i activate CI) 
 10 (CI_i inhibit EN) 
  9 (CI_i and HH_{i+-1} activate ptc) 
  7 (WG_i inhibit EN) 
  7 (SLP_i inhibit EN) 
  6 (CI_i inhibit en) 
  6 (CIA_i and SLP_i inhibit hh) 
  6 (EN_i inhibit CI) 
  6 (CIR_i inhibit hh) 
  6 (PTC_i inhibit HH) 
  6 (CI_i and SMO_i activate CIA) 
  6 (HH_i inhibit PTC) 
  6 (CIR_i inhibit en) 
  5 (CIA_i and SLP_i activate CI) 
  5 (CIA_i activate ptc) 
  5 (CIA_i activate PTC) 
  5 (CI_i and SMO_i inhibit CIR) 
  5 (PH_{i+-1} activate PTC) 
  5 (CI_i inhibit HH) 
  4 (HH_i activate SMO) 
  4 (CI_i activate SMO) 
  4 (CIR_i inhibit wg) 
  4 (EN_i inhibit PH) 
  3 (CIA_i and SLP_i activate ptc) 
  2 (CIR_i

### All clusters (filtered for supersets)

In [562]:
# Split up the groups of interactions and models in which we are knocking them out into evenly spaced groups
calls = []
n_samples = 100
step = 100
for cluster in clusters:
    # min call is to ensure that we get at most n_samples models.
    # If the cluster has less than n_samples models in it, then we'll just get all of them
    models = random.sample(clusters[cluster], k=(min(n_samples, len(clusters[cluster]))))
    interactions = list(filtered_interactions[cluster].keys())
    for start in range(0, len(interactions), step):
        calls.append(
                delayed(simulate_knockouts)(
                    cluster, 
                    interactions[start:start+step], 
                    models)
                )

In [563]:
with Parallel(n_jobs=45, verbose=10) as parallel:
    parallel_knockout_simulations_filtered = parallel(calls)    

[Parallel(n_jobs=45)]: Using backend LokyBackend with 45 concurrent workers.
[Parallel(n_jobs=45)]: Done   2 out of  11 | elapsed:    9.0s remaining:   40.5s
[Parallel(n_jobs=45)]: Done   4 out of  11 | elapsed:   11.2s remaining:   19.6s
[Parallel(n_jobs=45)]: Done   6 out of  11 | elapsed:   17.2s remaining:   14.3s
[Parallel(n_jobs=45)]: Done   8 out of  11 | elapsed:   24.0s remaining:    9.0s
[Parallel(n_jobs=45)]: Done  11 out of  11 | elapsed:   58.7s finished


In [587]:
# Merge together the split up simulations
knockout_simulations_filtered = {}
for key,scores in parallel_knockout_simulations_filtered:
    if key not in knockout_simulations: knockout_simulations_filtered[key] = {}
    # this is a weird bit of code but it just merges two dictionaries
    knockout_simulations_filtered[key] = {**knockout_simulations[key], **scores}

In [588]:
action_counts = {}

for (cluster, simulation_scores) in knockout_simulations_filtered.items():
    print('Cluster', cluster)
    print('===================')
  
    # sort by the combined score first
    sorted_scores = sorted(simulation_scores.items(), key=lambda _:_[1])[:10]
    
    for actions, (combined_score, synergy_score) in sorted_scores:
        print(f'{combined_score:.4f} {synergy_score:.4f}', end = " ")
        format_actions(sorted(actions))
        for action in actions:
            if action not in action_counts: action_counts[action] = []
            action_counts[action].append(cluster)

Cluster 0
0.8214 0.0097 (CI_i inhibit en) + (EN_i activate HH) 
0.8304 0.0037 (CI_i inhibit en) + (CIA_i and SLP_i activate wg) 
0.8322 0.0019 (CI_i inhibit en) + (CI_i and SMO_i activate ptc) 
0.8328 0.0013 (CI_i inhibit en) + (CIA_i and SLP_i activate CI) 
0.8533 -0.0255 (CI_i inhibit en) + (EN_i inhibit CI) 
0.8616 -0.0276 (CI_i inhibit en) + (CIA_i activate SMO) 
0.8653 0.0458 (CIR_i inhibit WG) + (EN_i activate HH) + (EN_i activate hh) 
0.8705 0.0387 (CIR_i inhibit WG) + (EN_i inhibit CI) + (EN_i activate hh) 
0.8883 0.0280 (CIR_i inhibit WG) + (EN_i inhibit CI) + (PH_i activate PTC) 
0.9016 0.0177 (CIA_i and SLP_i activate wg) + (CIR_i inhibit WG) + (EN_i activate HH) 
Cluster 1
0.6026 0.0937 (EN_i inhibit ci) + (HH_i activate SMO) + (SLP_i inhibit en) + (SLP_i and wg_i activate wg) 
0.6293 0.0670 (EN_i inhibit ci) + (HH_i activate SMO) + (SLP_i and wg_i activate wg) 
0.8658 0.0382 (CI_i inhibit hh) + (CIR_i inhibit SMO) 
0.8658 0.0465 (CIA_i activate ptc) + (CIR_i inhibit SMO) 


### Knockout unique interactions on other clusters

This was meant to be a sanity check that knocking out unique interactions from one cluster doesn't affect models from another cluster. Because we're using higher order interactions (pairs of interactions) we don't exactly see perfect null results. The reason is that if a interaction pair is unique, either one of the interactions in the pair could occur individually in other clusters.

In [ ]:
cross_cluster_knockouts = dict()
n_samples=50
for cluster_src in random.sample(unique_interactions.keys(), k=3):
    for cluster_other in random.sample(unique_interactions.keys(), k=3):
        if cluster_src != cluster_other:
            print('Unique interactions from cluster', cluster_src, 'knocked out in models from', cluster_other)
            models = random.sample(clusters[cluster_other], k=min(n_samples, len(clusters[cluster_other])))
            l = len(models)

            cross_cluster_knockouts[(cluster_src, cluster_other)] = dict()
            for action in unique_interactions[cluster_src]:
                cross_cluster_knockouts[(cluster_src, cluster_other)][action] = 0.0
                for model in models:
                    score = simulate_knockout(model, action)
                    cross_cluster_knockouts[(cluster_src, cluster_other)][action] += score/l


In [ ]:
for cluster_src, cluster_other in cross_cluster_knockouts:
    print('Unique interactions from cluster', cluster_src, 'knocked out in models from', cluster_other)
    for actions, avg_reward in sorted(cross_cluster_knockouts[(cluster_src, cluster_other)].items(), key=lambda _:_[1])[:10]:
        print(f'{avg_reward:.3f}', end = " ")
        format_actions(actions)